<a href="https://colab.research.google.com/github/Tachelgup/Python_base/blob/main/Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Type
import json
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class InvalidBicycleError(Exception): pass
class PermissionDeniedError(Exception): pass
class RentalNotFoundError(Exception): pass

class BicycleMeta(type):
    registry: Dict[str, Type['Bicycle']] = {}

    def __new__(cls, name, bases, classdict):
        new_class = super().__new__(cls, name, bases, classdict)
        if name != "Bicycle":
            cls.registry[name.lower()] = new_class
        return new_class

class Bicycle(ABC, metaclass=BicycleMeta):
    def __init__(self, bicycle_id: int, model: str, bike_type: str, hourly_rate: float, is_available: bool = True):
        self.__bicycle_id = bicycle_id
        self.__model = model
        self.__type = bike_type
        self.__hourly_rate = hourly_rate
        self.__is_available = is_available

    @abstractmethod
    def calculate_rental_cost(self, hours: int) -> float:
        pass

    def __str__(self):
        return f"Велосипед: {self.__model}, Тип: {self.__type}"

    def __lt__(self, other):
        return self.__hourly_rate < other.__hourly_rate

    def __gt__(self, other):
        return self.__hourly_rate > other.__hourly_rate

    def __eq__(self, other):
        return self.__model == other.__model and self.__type == other.__type

    @property
    def model(self): return self.__model
    @property
    def hourly_rate(self): return self.__hourly_rate
    @property
    def is_available(self): return self.__is_available

    @is_available.setter
    def is_available(self, value: bool): self.__is_available = value

    def to_dict(self):
        return {
            'bicycle_id': self.__bicycle_id,
            'model': self.__model,
            'type': self.__type,
            'hourly_rate': self.__hourly_rate,
            'is_available': self.__is_available
        }

    @classmethod
    def from_dict(cls, data):
        return cls(**data)

class CityBike(Bicycle):
    def __init__(self, *args, has_basket: bool = False, **kwargs):
        super().__init__(*args, **kwargs)
        self.has_basket = has_basket

    def calculate_rental_cost(self, hours: int) -> float:
        return self.hourly_rate * hours * (0.9 if hours > 3 else 1)

    def __str__(self):
        return f"Городской велосипед: {self.model}, Корзина: {self.has_basket}"

class MountainBike(Bicycle):
    def __init__(self, *args, suspension_type: str = "Hardtail", **kwargs):
        super().__init__(*args, **kwargs)
        self.suspension_type = suspension_type

    def calculate_rental_cost(self, hours: int) -> float:
        return self.hourly_rate * hours

    def __str__(self):
        return f"Горный велосипед: {self.model}, Амортизация: {self.suspension_type}"

class ElectricBike(Bicycle):
    def __init__(self, *args, battery_life: int = 5, **kwargs):
        super().__init__(*args, **kwargs)
        self.battery_life = battery_life

    def calculate_rental_cost(self, hours: int) -> float:
        return self.hourly_rate * hours + (5 if hours > 2 else 0)

    def __str__(self):
        return f"Электровелосипед: {self.model}, Заряд: {self.battery_life}ч"

class LoggingMixin:
    def log_action(self, message: str):
        logger.info(message)

class NotificationMixin:
    def send_notification(self, message: str):
        print(f"[Уведомление]: {message}")

class Rentable(ABC):
    @abstractmethod
    def rent_bicycle(self, bicycle: Bicycle, hours: int): pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self): pass

class Location:
    def __init__(self, address: str):
        self.address = address

class RentalStation(LoggingMixin, NotificationMixin):
    def __init__(self, station_id: int, location_info: Location, capacity: int):
        self.station_id = station_id
        self.location = location_info
        self.capacity = capacity
        self.bicycles: List[Bicycle] = []

    def add_bicycle(self, bike: Bicycle):
        if len(self.bicycles) < self.capacity:
            self.bicycles.append(bike)
            self.log_action(f"Велосипед {bike.model} добавлен на станцию.")
        else:
            raise Exception("Станция заполнена")

    def remove_bicycle(self, bike: Bicycle):
        self.bicycles.remove(bike)
        self.log_action(f"Велосипед {bike.model} удалён со станции.")

    def get_available_bicycles(self):
        return [b for b in self.bicycles if b.is_available]

class BicycleFactory:
    @staticmethod
    def create_bicycle(bike_type: str, *args, **kwargs) -> Bicycle:
        cls = BicycleMeta.registry.get(bike_type.lower())
        if not cls:
            raise InvalidBicycleError(f"Тип '{bike_type}' не зарегистрирован.")
        return cls(*args, **kwargs)

class RequestHandler:
    def __init__(self, successor=None):
        self.successor = successor

    def handle_request(self, request):
        if self.successor:
            return self.successor.handle_request(request)
        return "Отказано"

class StationOperator(RequestHandler):
    def handle_request(self, request):
        if request['level'] <= 1:
            return "Одобрено оператором"
        return super().handle_request(request)

class Manager(RequestHandler):
    def handle_request(self, request):
        if request['level'] <= 2:
            return "Одобрено менеджером"
        return super().handle_request(request)

class Admin(RequestHandler):
    def handle_request(self, request):
        return "Одобрено администратором"

class RentalProcess(ABC):
    def rent_bicycle(self, bicycle: Bicycle, hours: int):
        if not bicycle.is_available:
            raise RentalNotFoundError("Велосипед недоступен")
        self.prepare_rental()
        cost = bicycle.calculate_rental_cost(hours)
        self.confirm_rental(bicycle, cost)

    @abstractmethod
    def prepare_rental(self): pass

    @abstractmethod
    def confirm_rental(self, bicycle: Bicycle, cost: float): pass

class OnlineRentalProcess(RentalProcess):
    def prepare_rental(self):
        print("Подготовка онлайн аренды...")

    def confirm_rental(self, bicycle: Bicycle, cost: float):
        print(f"Онлайн аренда подтверждена. Стоимость: {cost}")

class OfflineRentalProcess(RentalProcess):
    def prepare_rental(self):
        print("Подготовка офлайн аренды...")

    def confirm_rental(self, bicycle: Bicycle, cost: float):
        print(f"Офлайн аренда подтверждена. Стоимость: {cost}")

def check_permissions(role_required):
    def decorator(func):
        def wrapper(user, *args, **kwargs):
            if user.get('role') != role_required:
                raise PermissionDeniedError("Недостаточно прав доступа")
            return func(user, *args, **kwargs)
        return wrapper
    return decorator

def save_bicycles_to_file(bicycles: List[Bicycle], filename: str):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump([bike.to_dict() for bike in bicycles], f, ensure_ascii=False)

def load_bicycles_from_file(filename: str) -> List[Bicycle]:
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return [BicycleFactory.create_bicycle(d['type'], **d) for d in data]
